In [1]:
import numpy as np
import math
from math import sin, cos, pi, factorial, exp
import random
from scipy import stats
from gym import Env, spaces #module that enables custom environment to conduct
from gym.spaces import Discrete, Box #discrete : discrete, box : continous!
from collections import deque

# Real Custom Environment

In [16]:
class CustomEnv:
    def __init__(self): 
        #Setting initializing
        self.W = 20 * (10 ** 6) #channel bandwidth
        self.I_max = 20 #maximum associated number
        self.D = 10000 #size of replay memory
        self.B_max = 100 #battery capacity
        self.M = 100 #data requests
        self.P_tx = 3 #transceiver power
        self.P_op = 10 #operation power
        self.P__c = 1 #swithcing power
        self.background_noise = -174 #backgroud noise
        self.alpha = 2 #path loss
        self.num_SBS = 5 #the number of SBS
        self.N = list(range(1, self.num_SBS + 1)) # SBS set # N = [1, 2, 3, 4, 5]
        self.T = 200
        self.t = np.int64(np.linspace(0, self.T, self.T)) # Time instant #t = (0, 2, ,,, , 199)
        self.time_interval = 1
        self.num_episodes = 200
        self.Distance = 50
        
        self.step_count = 0
        self.num_user = 0
        self.num_actions = 2 ** (self.num_SBS) # = 32
        self.state = None
        self.action_memory = None
        self.zeta = 0.9 #parameter of the utility function, which is to take control of the trade off between EE and delay
      
    ###action : SBS 5, action = 0,1
    ###mapping = {
        #          5 4 3 2 1             5 4 3 2 1
        #     0 : [0,0,0,0,0] #     1 : [0,0,0,0,1]
        #     2 : [0,0,0,1,0]#     3 : [0,0,0,1,1]
        #     4 : [0,0,1,0,0]#     5 : [0,0,1,0,1]
        #     6 : [0,0,1,1,0]#     7 : [0,0,1,1,1]
        #     8 : [0,1,0,0,0]#     9 : [0,1,0,0,1]
        #    10 : [0,1,0,1,0]#    11 : [0,1,0,1,1]
        #    12 : [0,1,1,0,0]#    13 : [0,1,1,0,1]
        #    14 : [0,1,1,1,0]#    15 : [0,1,1,1,1]
        #    16 : [1,0,0,0,0]#    17 : [1,0,0,0,1]
        #    18 : [1,0,0,1,0]#    19 : [1,0,0,1,1]
        #    20 : [1,0,1,0,0]#    21 : [1,0,1,0,1]
        #    22 : [1,0,1,1,0]#    23 : [1,0,1,1,1]
        #    24 : [1,1,0,0,0]#    25 : [1,1,0,0,1]
        #    26 : [1,1,0,1,0]#    27 : [1,1,0,1,1]
        #    28 : [1,1,1,0,0]#    29 : [1,1,1,0,1]
        #    30 : [1,1,1,1,0]#    31 : [1,1,1,1,1]
        ###}###
        
        
        bounds_lower = np.array([
            0,
            0,
            0,
            1e-1,
            1e-14,
            0,
            0,
            0,
            1e-1,
            1e-14,
            0,
            0,
            0,
            1e-1,
            1e-14,
            0,
            0,
            0,
            1e-1,
            1e-14,
            0,
            0,
            0,
            1e-1,
            1e-14,
            ])

        bounds_upper = np.array([
            7,
            self.B_max,
            1,
            1e+10,#total throughput max of SBS_1,
            1e+9,#total traffic delay max of SBS_1,
            7,
            self.B_max,
            1,
            1e+10,#total throughput max of SBS_2,
            1e+9,#total traffic delay max of SBS_2,
            7,
            self.B_max,
            1,
            1e+10,#total throughput max of SBS_3,
            1e+9,#total traffic delay max of SBS_3,
            7,
            self.B_max,
            1,
            1e+10,#total throughput max of SBS_4,
            1e+9,#total traffic delay max of SBS_4,
            7,
            self.B_max,
            1,
            1e+10,#total throughput max of SBS_5,
            1e+9,#total traffic delay max of SBS_5,
            ])

        self.action_space = spaces.Discrete(self.num_actions) # action size is here
        self.observation_space = spaces.Box(bounds_lower, bounds_upper, dtype=np.float32) # spaces.Discrete(2) # state size is here 

    def reset(self):
        reset_e = np.random.randint(0,200,self.num_SBS)
        self.state = [stats.poisson.rvs(mu = np.cos(self.t)+1)[reset_e][0], 0, np.random.randint(0,self.I_max + 1) / self.I_max, random.uniform(0,1e+10), (5 * self.M) / (random.uniform(1e-10,1e+10) * 2e+7), ########################
                      stats.poisson.rvs(mu = np.cos(self.t)+1)[reset_e][1], 0, np.random.randint(0,self.I_max + 1) / self.I_max, random.uniform(0,1e+10), (5 * self.M) / (random.uniform(1e-10,1e+10) * 2e+7),
                      stats.poisson.rvs(mu = np.cos(self.t)+1)[reset_e][2], 0, np.random.randint(0,self.I_max + 1) / self.I_max, random.uniform(0,1e+10), (5 * self.M) / (random.uniform(1e-10,1e+10) * 2e+7),
                      stats.poisson.rvs(mu = np.cos(self.t)+1)[reset_e][3], 0, np.random.randint(0,self.I_max + 1) / self.I_max, random.uniform(0,1e+10), (5 * self.M) / (random.uniform(1e-10,1e+10) * 2e+7),
                      stats.poisson.rvs(mu = np.cos(self.t)+1)[reset_e][4], 0, np.random.randint(0,self.I_max + 1) / self.I_max, random.uniform(0,1e+10), (5 * self.M) / (random.uniform(1e-10,1e+10) * 2e+7),
                      
                     ]        
        self.num_user = 0
        self.action_memory = deque(maxlen = self.T)
        self.step_count = 0
        return self.state
    
    def step(self, action):
        assert self.action_space.contains(action), "%r (%s) invalid"%(action, type(action))
        state = self.state
        reward = 0
        global action_memory, step_count # So as to use these variables in def energy_efficiency
        step_count = self.step_count
        action_memory = self.action_memory
        action_memory.append(action)
        self.num_user += self.num_users() #The number of users at step t
        e_1, b_1, l_1, th_1, d_1, e_2, b_2, l_2, th_2, d_2, e_3, b_3, l_3, th_3, d_3, e_4, b_4, l_4, th_4, d_4, e_5, b_5, l_5, th_5, d_5 = state
       
        # Update step
        i = np.random.randint(0,100,self.num_SBS)
        for p in range(self.num_SBS):
            state[p * 5] = stats.poisson.rvs(mu = np.cos(self.t)+1)[i][p] #update e in state
            state[p * 5 + 1] = self.battery_level(action)[p] #update b in state
            state[p * 5 + 2] = self.traffic_load(self.num_user)[0][p] #update l in state
            state[p * 5 + 3] = self.total_throughput(self.num_user, action)[1][p] #update r in state
            state[p * 5 + 4] = self.total_throughput(self.num_user, action)[2][p] #update d in state
 
        # Calculate reward : In this paper, there is no limit of zeta
        reward = self.zeta * self.energy_efficiency(self.num_user, action) - (1 - self.zeta) * self.total_throughput(self.num_user, action)[0]
        # Check if episode is done
        done = (self.step_count >= self.T)
        # Return step information
        info = {}
        return self.state, reward, done, info
    
    
    def render(self):
        # Implement visualization
        pass
 ##############################################################################################################   
    def battery_level(self, action): #return : battery_1, battery_2, battery_3, battery_4, battery_5
        state = self.state
        battery_1 = min(state[1] - (self.P_tx * ((action & 0b00001) / 0b00001)) + state[0], self.B_max)
        battery_2 = min(state[6] - (self.P_tx * ((action & 0b00010) / 0b00010)) + state[5], self.B_max)
        battery_3 = min(state[11] - (self.P_tx * ((action & 0b00100) / 0b00100)) + state[10], self.B_max)
        battery_4 = min(state[16] - (self.P_tx * ((action & 0b01000) / 0b01000)) + state[15], self.B_max)
        battery_5 = min(state[21] - (self.P_tx * ((action & 0b10000) / 0b10000)) + state[20], self.B_max)
        return battery_1, battery_2, battery_3, battery_4, battery_5
    
    def num_users(self):#Increasement of users at step #return : num_users
        user_come = stats.poisson.rvs(mu = np.sin(self.t)+1)[np.random.randint(0,200)] 
        user_leave = np.floor(stats.expon.rvs(scale = 1, size = self.T))[np.random.randint(0,200)] 
        num_users = user_come - user_leave
        if num_users < 0:
            num_users = 0
        return num_users 
    
    def traffic_load(self, num_user): #return : traffic load, user set, dis_dis
        #Deciding location of SBS
        loc_SBS = [] 
        for l_SBS in range(self.num_SBS):
            radius = np.random.uniform(0,self.Distance)
            radian = random.random()
            x_SBS = radius * math.cos(2 * math.pi * radian)
            y_SBS = radius * math.sin(2 * math.pi * radian)
            loc_SBS.append([x_SBS,y_SBS])
        
        #Deciding location of User
        loc_user = []
        for l in range(int(num_user)):
            radius_ = np.random.uniform(0,self.Distance)
            radian_ = random.random()
            x_user = radius_ * math.cos(2 * math.pi * radian_)
            y_user = radius_ * math.sin(2 * math.pi * radian_)
            loc_user.append([x_user,y_user])
        
        def distance(x1, y1, x2, y2):
            result = math.sqrt(math.pow(x1 - x2, 2) + math.pow(y1 - y2, 2))
            return result

        #Distance between SBSs and users. In other words, that should be a matrix form.
        dis_dis = []
        for dis in range(int(num_user)):
            jth_jth = []
            for jth in range(self.num_SBS): 
                jth_jth.append(distance(loc_SBS[jth][0],loc_SBS[jth][1],loc_user[dis][0],loc_user[dis][1]))
            dis_dis.append(jth_jth)
 
        #The set of SBSs that is the nearest to each user.
        j_star = [0 for _ in range(int(num_user))]
        for y in range(int(num_user)):
            j_star[y] = dis_dis[y].index(min(dis_dis[y]))
        
        def kth_largest_number(arr, K):
            unique_nums = set(arr)
            sorted_nums = sorted(unique_nums, reverse=False) #
            return sorted_nums[K-1]
        
        def j_kth_star(k):
            j_kth_star = []
            j_second_star = [0 for _ in range(int(num_user))]
            for S_ in range(int(num_user)):
                j_second_star[S_] = dis_dis[S_].index(kth_largest_number(dis_dis[S_], k))
            j_kth_star.append(j_second_star)     
            return j_kth_star

        #If user set of a SBS exceeds 20.
        user_set = [0 for _ in range(self.num_SBS)]
        for u in range(self.num_SBS):
                user_set[u] = [j_index for j_index, value in enumerate(j_star) if value == u] 
                if len(user_set[u]) > self.I_max:
                    user_set[u] = [j_index for j_index, value in enumerate(j_kth_star(2)) if value == u]  
                    if len(user_set[u]) > self.I_max:
                        user_set[u] = [j_index for j_index, value in enumerate(j_kth_star(3)) if value == u]
                        if len(user_set[u]) > self.I_max:
                            user_set[u] = [j_index for j_index, value in enumerate(j_kth_star(4)) if value == u]
                            if len(user_set[u]) > self.I_max:
                                user_set[u] = [j_index for j_index, value in enumerate(j_kth_star(5)) if value == u] 

        abs_user_set = [0 for _ in range(self.num_SBS)]
        for a in range(self.num_SBS):
            abs_user_set[a] = len(user_set[a])
        
        traffic_load = [0 for _ in range(self.num_SBS)]
        for t in range(self.num_SBS):
            traffic_load[t] = abs_user_set[t] / self.I_max

        return np.array(traffic_load), np.array(user_set), np.array(dis_dis)

    def total_throughput(self, num_user, action): #return : total_delay, total_throughput, traffic_delay
        u_set = self.traffic_load(int(num_user))[1]
        #Implementation of rho_i_t
        act = []
        c = 0
        for o in range(int(num_user)):
            if o in u_set[0]:
                c = ((action & 0b00001) / 0b00001)
            elif o in u_set[1]:
                c = ((action & 0b00010) / 0b00010)
            elif o in u_set[2]:
                c = ((action & 0b00100) / 0b00100)
            elif o in u_set[3]:
                c = ((action & 0b01000) / 0b01000)
            elif o in u_set[4]:
                c = ((action & 0b10000) / 0b10000)
            act.append(c)
                
        ##Numerator of sinr 
        signal = []
        #for s in range(num_user)
        for s in range(int(num_user)):   
            s_ = []
            for si in range(self.num_SBS):
                s_.append((self.P_tx) * (np.exp(1) * act[s] * (self.traffic_load(int(num_user))[2][s][si])))            
            signal.append(s_)
        
        ##Implementation path loss to denominator of sinr
        signal_alpha = []
        for s in range(int(num_user)):   
            ph = []
            for si in range(self.num_SBS):
                ph.append((self.P_tx) * (np.exp(1) * act[s] * (self.traffic_load(int(num_user))[2][s][si] ** (-self.alpha))))            
            signal_alpha.append(ph)
    
        ##Sum
        signal_denominator = [0 for _ in range(int(num_user))]
        for n_ in range(int(num_user)):
            for no in range(self.num_SBS):
                signal_denominator[n_] = signal_denominator[n_] + signal_alpha[n_][no]
   
        sinr = []
        for z in range(int(num_user)):
            sinr_ = []
            for q in range(self.num_SBS):
                sinr_.append((signal[z][q]) / ((signal_denominator[z] - (signal_alpha[z][q])) + (2 * 10 **(-13.4))))
            sinr.append(sinr_)
    
        throughput = []
        for i in range(int(num_user)):
            t_t_j = []
            for j in range(self.num_SBS): 
                t_t_j.append(self.time_interval * self.W * np.log2(1 + sinr[i][j]))
            throughput.append(t_t_j)
        
        total_throughput = [0 for _ in range(self.num_SBS)]
        for t in range(self.num_SBS):
            for j in range(int(num_user)):
                total_throughput[t] = total_throughput[t] + throughput[j][t]
                     
        data_rate = []
        for dy in range(int(num_user)):
            data_rate_ = []
            for dz in range(self.num_SBS): 
                data_rate_.append(self.W * throughput[dy][dz])
            data_rate.append(data_rate_)
            
        traffic_delay = [0 for _ in range(self.num_SBS)]
        for t in range(self.num_SBS):
            for z in range(int(num_user)):
                if data_rate[z][t] >= 1e+10:
                    data_rate[z][t] = 1e+10
                elif data_rate[z][t] <= 1e-10:
                    data_rate[z][t] = 1e-10
                traffic_delay[t] = traffic_delay[t] + (self.M / data_rate[z][t])   

        total_delay = 0
        for t in range(self.num_SBS):
            total_delay += traffic_delay[t]
        
        return np.array(total_delay), np.array(total_throughput), np.array(traffic_delay)
    
    def energy_efficiency(self, num_user, action):#return : np.array(ee)
        power_serving = [0 for _ in range(self.num_SBS)]
        power_serving[0] = ((self.traffic_load(int(num_user))[0][0] * self.P_op) + self.P_tx) * ((action & 0b00001) / 0b00001)
        power_serving[1] = ((self.traffic_load(int(num_user))[0][1] * self.P_op) + self.P_tx) * ((action & 0b00010) / 0b00010)
        power_serving[2] = ((self.traffic_load(int(num_user))[0][2] * self.P_op) + self.P_tx) * ((action & 0b00100) / 0b00100)
        power_serving[3] = ((self.traffic_load(int(num_user))[0][3] * self.P_op) + self.P_tx) * ((action & 0b01000) / 0b01000)
        power_serving[4] = ((self.traffic_load(int(num_user))[0][4] * self.P_op) + self.P_tx) * ((action & 0b10000) / 0b10000)
                 
        beta = [0 for _ in range(self.num_SBS)]
        if (((action_memory[self.step_count]) & 0b00001) / 0b00001) + (((action_memory[self.step_count - 1]) & 0b00001) / 0b00001) == 1:
            beta[0] = 1#position transition
        else:
            beta[0] = 0
        if (((action_memory[self.step_count]) & 0b00010) / 0b00010) + (((action_memory[self.step_count - 1]) & 0b00010) / 0b00010) == 1:
            beta[1] = 1
        else:
            beta[1] = 0            
        if (((action_memory[self.step_count]) & 0b00100) / 0b00100) + (((action_memory[self.step_count - 1]) & 0b00100) / 0b00100) == 1:
            beta[2] = 1
        else:
            beta[2] = 0            
        if (((action_memory[self.step_count]) & 0b01000) / 0b01000) + (((action_memory[self.step_count - 1]) & 0b01000) / 0b01000) == 1:
            beta[3] = 1
        else:
            beta[3] = 0            
        if (((action_memory[self.step_count]) & 0b10000) / 0b10000) + (((action_memory[self.step_count - 1]) & 0b10000) / 0b10000) == 1:
            beta[4] = 1     
        else:
            beta[4] = 0
        
        power_switch = [0 for _ in range(self.num_SBS)]
        for p in range(self.num_SBS):
            power_switch[p] = beta[p] * self.P__c
        
        total_p_consumption = [0 for _ in range(self.num_SBS)]
        for t in range(self.num_SBS):
            total_p_consumption[t] = [power_switch[t] + power_serving[t]]
        total_power_consumption = np.sum(total_p_consumption)
        
        total_j_throughput = 0
        for t in range(self.num_SBS):
            total_j_throughput += self.total_throughput(int(num_user), action)[1][t]
            
        ee = [total_j_throughput /  total_power_consumption]
        
        return np.array(ee) 

In [17]:
env = CustomEnv()

In [18]:
env.reset()
env.step(9)

([0,
  -3.0,
  0.0,
  0.0,
  1e-08,
  3,
  3.0,
  0.0,
  0.0,
  1000000000000.0,
  0,
  0.0,
  0.05,
  201614530.81602463,
  1000000000000.0,
  4,
  1.0,
  0.05,
  0.0,
  1e-08,
  0,
  0.0,
  0.0,
  242910217.0452812,
  1e-08],
 array([-4.99906036e+11]),
 False,
 {})

In [19]:
env.action_space.sample()

26

In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [21]:
states = env.observation_space.shape
actions = env.action_space.n

In [22]:
states

(25,)

In [23]:
actions

32

In [24]:
def build_model(states, actions):
    model = Sequential()
    #model.add(Dense(128, input_shape = (25,), activation = 'tanh')) 
    model.add(Flatten(input_shape = (1,25))) 
    #Normally, input layer has used "Flatten". However, my input_shape is (25,1).
    model.add(Dense(64, activation = 'tanh'))
    model.add(Dense(32, activation = 'tanh'))
    model.add(Dense(actions, activation = 'sigmoid'))
    return model  

In [25]:
del model
#Why??!!!!

NameError: name 'model' is not defined

In [26]:
model = build_model(states, actions)

In [27]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 25)                0         
_________________________________________________________________
dense (Dense)                (None, 64)                1664      
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 32)                1056      
Total params: 4,800
Trainable params: 4,800
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [19]:
def build_agent(model, acitons):
    K =200
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit = K, window_length = 1)
    #"window_length" decides how many observations would be admitted as a "state"
    #DQN on Atari같은 경우는 공의 속도를 알기 위해 window 4개를 1개의 state로 인정했지
    dqn = DQNAgent(model = model, memory = memory, policy = policy, nb_actions = actions, 
                   nb_steps_warmup = K, 
                   target_model_update = 1e-3)
    #nb_steps_warmup : experience replay쓰기전에 얼마나 기다릴 것인가(적당한 batch size 생성위해)
    #target_model_update : target net 얼마나 고정시킬 것이냐
    return dqn

In [ ]:
K = 100
dqn = build_agent(model, actions)
dqn.compile(Adam(lr = 1e-3), metrics = ['mse'])
#compile(self, optimizer, metrics = [])
dqn.fit(env, nb_steps = K, visualize = False, verbose = 1)
#nb_steps : number of training steps to be performed

Training for 200 steps ...
Interval 1 (0 steps performed)
   36/10000 [..............................] - ETA: 4:20:24 - reward: -4318867041507.5410